In [1]:
import geopandas as gpd
import os

In [2]:
def create_land_buffer(
    country_gdf, harbor_gdf, buffer_country, buffer_harbor
):
    """
    Creates a buffer around harbors and countries and saves it GeoJSON.
    Arguments:
        country_gdf: geodataframe all countries
        harbor_gdf: geodataframe all harbors globally
        buffer_harbor: size buffer around harbors (km)
        buffer_country: size buffer around countries (km)
    Returns:
        None, but saves GeoJSON
    """
    assert country_gdf.crs == harbor_gdf.crs
    assert harbor_gdf.is_valid.all() and country_gdf.is_valid.all()
    # conversion https://www.usna.edu/Users/oceano/pguth/md_help/html/approx_equivalents.htm
    # 1 degree = 111 km
    # 1 km = 0.009 degrees
    # This is usually accurate enough, but should not be used for anything near the poles
    buffer_country_deg = buffer_country * 0.009
    buffer_harbor_deg = buffer_harbor * 0.009
    # Dissolve it
    countries_dissolved = gpd.GeoDataFrame(country_gdf.dissolve()["geometry"])
    harbors_dissolved = gpd.GeoDataFrame(harbor_gdf.dissolve()["geometry"])
    print("Create the buffers")    
    buffered_harbors = gpd.GeoDataFrame(harbors_dissolved.buffer(buffer_harbor_deg))
    buffered_harbors.columns = ["geometry"]
    buffered_countries = gpd.GeoDataFrame(
        countries_dissolved.buffer(buffer_country_deg)
    )
    buffered_countries.columns = ["geometry"]
    # Combine the two bufferes
    buffer_both = buffered_countries.union(buffered_harbors)
    print("Substract the countries from the buffer")
    buffer_diff = gpd.GeoDataFrame(buffer_both.difference(countries_dissolved))
    buffer_diff.columns = ["geometry"]
    buffer_diff = buffer_diff.dissolve()
    print("Saving")
    buffer_diff.to_file(
        "results"
        + os.sep
        + "harbor_{}km_coast_{}km_buffer_no_metadata.gpkg".format(buffer_harbor, buffer_country),
        driver="GPKG")


In [3]:
def buffer_with_meta_data(buffer, meta, buffer_country, buffer_harbor):
    """
    Overlays the buffer with a file containing metadata for those regions. 
    """
    print("overlap")
    overlap_df = gpd.overlay(meta, buffer, how="intersection")
    # Iterate over the columns of each dataframe
    for col in meta.columns:
        # Add the column to the overlap dataframe with a suffix indicating the source dataframe
        overlap_df[col + '_1'] = meta[col]
    # Filter the overlap dataframe to only include the intersecting polygons
    overlap_df = overlap_df[overlap_df['geometry'].notnull()]
    # only keep the columns we need
    overlap_df = overlap_df[["SOVEREIGN1", "geometry"]]
    print("Saving")
    overlap_df.to_file(
        "results"
        + os.sep
        + "harbor_{}km_coast_{}km_buffer_with_metadata.gpkg".format(buffer_harbor, buffer_country),
        driver="GPKG")

In [4]:
harbors = gpd.read_file("global_harbors.json")
harbors.head(2)

,id,portname,code,prttype,prtsize,status,maxdepth,maxlength,annualcapacitymt,humuse,...,country,lastcheckdate,remarks,url_lca,source,createdate,updatedate,geonameid,gdb_geomattr_data,geometry
0,wld_trs_ports_wfp.14314,Watsi-Genge,None,River,Very Small,Unknown,NaN,NaN,NaN,Unknown,...,Democratic Republic of the Congo,NaT,None,None,None,2021-02-24 11:52:47.493000+00:00,2021-02-24 11:52:47.493000+00:00,204280,None,POINT (20.62966 -0.94560)
1,wld_trs_ports_wfp.14315,Charlotte (Skidegate),CASKI,Sea,Unknown,Open,NaN,NaN,NaN,Unknown,...,Canada,NaT,None,None,None,2021-02-24 11:52:47.493000+00:00,2021-02-24 11:52:47.493000+00:00,6148858,None,POINT (-132.00969 53.24742)


In [5]:
countries = gpd.read_file("Countries" + os.sep + "ne_50m_admin_0_countries.shp")
countries.head(2)

,featurecla,scalerank,LABELRANK,SOVEREIGNT,SOV_A3,ADM0_DIF,LEVEL,TYPE,TLC,ADMIN,...,FCLASS_TR,FCLASS_ID,FCLASS_PL,FCLASS_GR,FCLASS_IT,FCLASS_NL,FCLASS_SE,FCLASS_BD,FCLASS_UA,geometry
0,Admin-0 country,1,3,Zimbabwe,ZWE,0,2,Sovereign country,1,Zimbabwe,...,None,None,None,None,None,None,None,None,None,"POLYGON ((31.28789 -22.40205, 31.19727 -22.344..."
1,Admin-0 country,1,3,Zambia,ZMB,0,2,Sovereign country,1,Zambia,...,None,None,None,None,None,None,None,None,None,"POLYGON ((30.39609 -15.64307, 30.25068 -15.643..."


In [6]:
# EEZ files are too large for Github. They have to be downloaded here: 
# https://www.marineregions.org/downloads.php
eez = gpd.read_file("EEZ" + os.sep + "eez_v11.shp")
eez.head(2)

,GEONAME,TERRITORY1,ISO_TER1,SOVEREIGN1,geometry
0,American Samoa Exclusive Economic Zone,American Samoa,ASM,United States,"POLYGON ((-166.64112 -17.55527, -166.64194 -17..."
1,Ascension Exclusive Economic Zone,Ascension,SHN,United Kingdom,"POLYGON ((-10.93328 -7.88745, -10.93324 -7.889..."


In [7]:
buffer_country = 2.5
buffer_harbor = 50

In [8]:
create_land_buffer(countries, harbors, buffer_country, buffer_harbor)

Create the buffers


/tmp/ipykernel_15612/697092470.py:26: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buffered_harbors = gpd.GeoDataFrame(harbors_dissolved.buffer(buffer_harbor_deg))
/tmp/ipykernel_15612/697092470.py:29: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  countries_dissolved.buffer(buffer_country_deg)


Substract the countries from the buffer
Saving


In [9]:
buffer = gpd.read_file("results" + os.sep + "harbor_50km_coast_2.5km_buffer_no_metadata.gpkg")
buffer

,geometry
0,"MULTIPOLYGON (((-163.81698 -82.85067, -163.817..."


In [10]:
buffer_with_meta_data(buffer, eez, buffer_country, buffer_harbor)

overlap


/tmp/ipykernel_15612/2570273260.py:6: UserWarning: `keep_geom_type=True` in overlay resulted in 3188 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  overlap_df = gpd.overlay(meta, buffer, how="intersection")


Saving
